
# 🧪 اختبار أداء نموذج توصية النباتات

هذا الملف يهدف إلى تقييم أداء النموذج من حيث:
- سرعة الاستجابة.
- الاستقرار عبر إرسال عدة طلبات.



## 🧠 تدريب النموذج

نقوم بتحميل البيانات وتدريب نموذج `RandomForestClassifier` على سمات `humidity` و `co2` ثم نحفظ النموذج للاستخدام لاحقًا.



## 🌐 بناء واجهة API باستخدام Flask

نستخدم Flask لتقديم نموذج التوصية عبر نقطة نهاية `/recommend`.


In [ ]:

import requests
import time

# عدد الطلبات التي نريد إرسالها لاختبار الأداء
num_requests = 20

# بيانات إدخال تجريبية
test_payload = {
    "co2": 800,
    "humidity": 60
}

# نستخدم localhost بدلاً من ngrok
url = "http://127.0.0.1:5000/recommend"  # تأكد من تشغيل السيرفر محليًا

response_times = []

for i in range(num_requests):
    start_time = time.time()
    response = requests.post(url, json=test_payload)
    end_time = time.time()
    
    duration = end_time - start_time
    response_times.append(duration)
    print(f"Request {i+1}: {duration:.4f} seconds - Status Code: {response.status_code}")

# حساب متوسط زمن الاستجابة
avg_time = sum(response_times) / len(response_times)
print(f"Average Response Time over {num_requests} requests: {avg_time:.4f} seconds")



## 📈 اختبار الأداء

نقوم بإرسال 20 طلبًا إلى السيرفر وقياس زمن الاستجابة لكل طلب، ثم نحسب المتوسط.


In [ ]:

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(range(1, len(response_times) + 1), response_times, marker='o')
plt.title('Response Time per Request')
plt.xlabel('Request Number')
plt.ylabel('Response Time (seconds)')
plt.grid(True)
plt.show()



## 📊 رسم بياني لزمن الاستجابة

يعرض هذا الرسم زمن الاستجابة لكل طلب لتقييم الاستقرار والأداء.


In [1]:
!pip install pyngrok scikit-learn pandas joblib flask


In [2]:
from pyngrok import ngrok
ngrok.set_auth_token("2vdEEy402NGHkUfC0LQHmsAzDb6_6nsks9LCY8nfZWhtedQkA")


In [3]:
import pandas as pd
import numpy as np
import joblib
from flask import Flask, request, jsonify
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, top_k_accuracy_score


In [4]:
from google.colab import files
uploaded = files.upload()

df = pd.read_csv(next(iter(uploaded)))
df.head()


Saving updated_plant_data.csv to updated_plant_data.csv


,Unnamed: 0,common_name,family,categories,origin,climate,zone,img_url,suitable_temperature,estimated_humidity,estimated_co2
0,0,Janet Craig,Liliaceae,Dracaena,Cultivar,Tropical,"[11,10]",http://www.tropicopia.com/house-plant/thumbnai...,-1°C to 10°C,72,668
1,1,Lady palm,Arecaceae,Palm,China,Subtropical,"[11, 9]",http://www.tropicopia.com/house-plant/thumbnai...,-7°C to 10°C,62,557
2,2,"Tailflower , Wax flower",Araceae,Anthurium,Cultivar,Tropical,[11],http://www.tropicopia.com/house-plant/thumbnai...,4°C to 10°C,84,609
3,3,Elephant ear,Araceae,Other,Hybrid,Tropical humid,[11],http://www.tropicopia.com/house-plant/thumbnai...,4°C to 10°C,76,607
4,4,Malaysian Dracaena,Liliaceae,Dracaena,Cultivar,Tropical,"[11,10]",http://www.tropicopia.com/house-plant/thumbnai...,-1°C to 10°C,62,593


In [5]:
plant_counts = df['common_name'].value_counts()
filtered_df = df[df['common_name'].isin(plant_counts[plant_counts >= 5].index)]
print(f"Original plants: {len(plant_counts)}, After filtering: {filtered_df['common_name'].nunique()}")


Original plants: 132, After filtering: 15


In [6]:
X = filtered_df[['estimated_humidity', 'estimated_co2']]
y = filtered_df['common_name']

le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

joblib.dump(model, 'plant_recommender_model_humidity_co2.pkl')
joblib.dump(le, 'label_encoder_humidity_co2.pkl')


['label_encoder_humidity_co2.pkl']

In [7]:
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
top3 = top_k_accuracy_score(y_test, model.predict_proba(X_test), k=3, labels=model.classes_)

print(f"Accuracy: {acc:.2f}")
print("Top-3 Accuracy:", top3)
print("\nClassification Report:")
print(classification_report(
    y_test,
    y_pred,
    labels=np.unique(y_test),
    target_names=le.inverse_transform(np.unique(y_test)).astype(str)
)
)


Accuracy: 0.21
Top-3 Accuracy: 0.4642857142857143

Classification Report:
                       precision    recall  f1-score   support

  Chinese Evergreen         0.46      0.55      0.50        11
             Croton         0.00      0.00      0.00         2
            Dumcane         0.00      0.00      0.00         2
        English ivy         0.00      0.00      0.00         1
           Guzmania         0.00      0.00      0.00         1
        Janet Craig         0.00      0.00      0.00         2
         Peace lily         0.00      0.00      0.00         2
       Rubber plant         0.00      0.00      0.00         2
        Snake plant         0.00      0.00      0.00         1
      Umbella plant         0.00      0.00      0.00         2
        Weeping fig         0.00      0.00      0.00         2

            micro avg       0.30      0.21      0.25        28
            macro avg       0.04      0.05      0.05        28
         weighted avg       0.18      0.21

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
app = Flask(__name__)

model = joblib.load('plant_recommender_model_humidity_co2.pkl')
le = joblib.load('label_encoder_humidity_co2.pkl')

@app.route('/recommend', methods=['POST'])
def recommend_plant():
    try:
        data = request.get_json()
        humidity = data['humidity']
        co2 = data['co2']
        probs = model.predict_proba([[humidity, co2]])[0]
        top3_indices = np.argsort(probs)[-3:][::-1]
        top3_plants = le.inverse_transform(top3_indices).tolist()
        return jsonify({"top_3_recommendations": top3_plants})
    except Exception as e:
        return jsonify({"error": str(e)}), 400


public_url = ngrok.connect(5000)
print("🔗 Public URL:", public_url)

app.run()


🔗 Public URL: NgrokTunnel: "https://caf5-35-233-159-125.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:

import requests
import time

# عدد الطلبات التي نريد إرسالها لاختبار الأداء
num_requests = 20

# بيانات إدخال تجريبية
test_payload = {
    "co2": 800,
    "humidity": 60
}

# نستخدم localhost بدلاً من ngrok
url = "http://127.0.0.1:5000/recommend"  # تأكد من تشغيل السيرفر محليًا

response_times = []

for i in range(num_requests):
    start_time = time.time()
    response = requests.post(url, json=test_payload)
    end_time = time.time()
    
    duration = end_time - start_time
    response_times.append(duration)
    print(f"Request {i+1}: {duration:.4f} seconds - Status Code: {response.status_code}")

# حساب متوسط زمن الاستجابة
avg_time = sum(response_times) / len(response_times)
print(f"Average Response Time over {num_requests} requests: {avg_time:.4f} seconds")


In [ ]:

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(range(1, len(response_times) + 1), response_times, marker='o')
plt.title('Response Time per Request')
plt.xlabel('Request Number')
plt.ylabel('Response Time (seconds)')
plt.grid(True)
plt.show()
